In [27]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [28]:

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS demo 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [13]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [14]:
import os,glob
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

C:\Users\Zain\dataengineer\apache-cassandra


In [15]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [32]:
query = """CREATE TABLE demo.music_app_history_demo (session_id bigint , 
    item_in_session int, 
    artist text, 
    song_title text,
    song_length float, 
    PRIMARY KEY (session_id, item_in_session))"""
session.execute(query);

In [38]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO demo.music_app_history_demo (session_id, item_in_session ,artist ,song_title ,song_length)"
        query = query + " VALUES (%s, %s, %s,%s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5] )))

In [40]:
query = 'SELECT * FROM demo.music_app_history_demo '
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.session_id, row.item_in_session, row.artist, row.song_title, row.song_length)

990 1 Frou Frou Breathe In 277.3415832519531
990 2 Justin Timberlake What Goes Around...Comes Around 313.07708740234375
1041 0 David Bowie Sorrow (1997 Digital Remaster) 174.4191436767578
1041 1 J Dilla Won't Do 232.46322631835938
1041 2 Hot Hot Heat In Cairo 295.0264892578125
1041 3 Norther We Rock 236.85179138183594
1041 4 Postal Service Nothing Better (Album) 226.61178588867188
1041 5 311 Something Out Of Nothing 264.5677185058594
1041 6 Jonas Brothers When You Look Me In The Eyes 249.67791748046875
1041 7 Platero Y Tu Cantalojas 368.2999572753906
1041 8 Martina McBride In My Daughter's Eyes 193.802001953125
1041 9 Ben Folds Five Smoke 292.23138427734375
1041 10 Seine So far so long 173.66159057617188
1041 11 Shinedown What A Shame (Album Version) 258.9775085449219
1041 12 Iva Vanity Case (nouvelle version) 205.32200622558594
1041 13 Cathy Davey Hammerhead (Explicit) 273.5277099609375
1041 14 Jorge Reyes Valencia Hekura 127.81668853759766
1041 15 Sonic Youth Candle 299.1015930175781

716 53 Jokeren Kvinde Din - MÃÂ¸gluder 239.4901580810547
716 54 Ween Demon Sweat 250.1742401123047
716 55 King Diamond No Presents For Christmas 408.4762878417969
716 56 Mint Condition Somethin' (feat. Phonte from Little Brother) 279.065673828125
716 57 Delorean Complexity Reducer 272.6395568847656
716 58 MylÃÂ¨ne Farmer Porno Graphique 257.227294921875
716 59 Green Day Horseshoes And Handgrenades (Album Version) 194.06321716308594
716 60 The New Pornographers Adventures In Solitude 256.4175109863281
716 61 Plies Goons Lurkin (Explicit Album Version) 231.62730407714844
925 1 Parov Stelar Good Bye Emily (feat. Gabriella Hanninen) 203.65016174316406
1000 0 Peter Sellers She Loves You 142.3930206298828
1000 1 Hellhammer Maniac 241.7105712890625
760 0 Filterfunk S.O.S. (Message In A Bottle) (Hi_Tack Radio Edit) 173.03465270996094
760 1 Selena Gomez & The Scene Stop & Erase 172.61668395996094
896 1 Bersuit Vergarabat La Flor De Mis Heridas 259.6305847167969
896 2 Black Sheep The Choice Is

1065 9 Linkin Park Figure.09 (Album Version) 197.51138305664062
1065 10 Underworld Teardrop 329.53424072265625
1065 11 John Mayer Heartbreak Warfare 269.7138366699219
1065 12 Thousand Foot Krutch Step To Me 180.58404541015625
1065 13 Claudio Minore Non Saprai Mai 282.01751708984375
1065 14 Wilco War on war 227.6566925048828
1065 15 Mission Of Burma Red 218.2003631591797
1065 16 Radiohead Sulk 223.00689697265625
1065 17 Hoobastank To Be With You 242.78158569335938
1065 18 Bright Eyes Poison Oak (Album Version) 279.8754577636719
1065 19 Procol Harum Grand Hotel 368.5611877441406
1065 20 Radiohead Planet Telex 259.2909851074219
1065 21 Alliance Ethnik SinceritÃÂ© Et Jalousie 195.9440460205078
1065 22 Muse New Born 365.4526062011719
1065 23 Steps 5_ 6_ 7_ 8 201.7693328857422
1065 24 John Mayer The Heart Of Life 198.19056701660156
1065 25 David Gray Draw The Line 263.3399353027344
1065 26 Tub Ring Invalid 233.69097900390625
1065 27 Era Cathar Rhythm 200.5677032470703
1065 29 + / - {Plus/Mi

486 36 Thee Headcoats Monkey's Paw 160.26077270507812
486 37 matchbox twenty Disease (Album Version) 218.72280883789062
486 38 Primal Scream Movin' On Up 226.27220153808594
486 39 DJ Krush & Toshinori Kondo Shoh-ka 279.9015808105469
486 40 Mindless Faith Independence Day (version) 312.4762878417969
486 41 Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile) 277.1587219238281
486 42 Old 97's Victoria (LP Version) 231.28770446777344
486 43 The Dangerous Summer Wake Up 218.3309783935547
486 45 Randy Travis Hard Rock Bottom Of Your Heart (Single Remix Remastered Version) 237.03465270996094
486 46 M.I.A. Paper Planes 206.1317901611328
486 47 The Strokes Last Nite 197.53750610351562
486 48 Justice Let There Be Light (album version) 295.9407653808594
486 50 Guns N' Roses Sympathy For The Devil 455.5227966308594
486 51 Widespread Panic Papa's Home 400.7962951660156
486 52 Testament Afterlife 253.6485290527343

817 43 La Monja Enana VillapingÃÂ¼ino 144.43057250976562
817 44 The White Stripes Seven Nation Army (Album Version) 231.81016540527344
817 45 Silent Stream Of Godless Elegy TvÃÂ¡rÃÂ­ v TvÃÂ¡r 412.8387451171875
817 46 Florence + The Machine I'm Not Calling You A Liar 185.31219482421875
817 47 Iam Une Femme Seule 278.229736328125
817 48 Adina Howard Horny For Your Love (LP Version) 236.40771484375
224 1 DeGarmo & Key I'm Accepted (The Pledge Album Version) 236.486083984375
391 1 Method Man The Motto 204.64280700683594
391 2 The Stanley Brothers I'm A Man Of Constant Sorrow 179.6958770751953
391 3 Dexter Freebish Deeper 210.54649353027344
806 1 My Chemical Romance Helena (So Long & Goodnight) (Album Version) 203.04934692382812
806 2 David Gray Sail Away 313.59954833984375
806 3 Lisa Hannigan Lille 246.961181640625
806 4 Metallica / Marianne Faithfull The Memory Remains 279.117919921875
806 5 Bon Jovi Livin' On A Prayer 250.69668579101562
806 6 Barry Tuckwell/Academy of St Martin-in-th

...

255 19 Mae We're So Far Away 230.13832092285156
255 20 Helloween Where The Rain Grows 309.2109680175781
255 21 Justin Timberlake SeÃÂ±orita 294.8697509765625
255 22 Me First And The Gimme Gimmes Good Bye Earl 145.78892517089844
255 23 Usher Hey Daddy (Daddy's Home) 224.10403442382812
255 24 John Mayer Waiting On The World To Change 201.1685333251953
255 25 The Cute Lepers Prove It 195.5260772705078
255 26 Nando Reis RelicÃÂ¡rio 245.96853637695312
255 27 Kid Cudi / Ratatat Alive (nightmare) 246.72607421875
255 28 Huey Lewis & The News I Want A New Drug 286.43218994140625
255 29 Randy Houser Boots On 171.88525390625
255 30 Bo Hansson Lothlorien (2002 Digital Remaster) 240.63955688476562
255 31 Karen Clark Sheard Good 220.81260681152344
255 32 Alejandro Sanz Sin que se note 266.8403625488281
255 35 The New Pornographers Go Places 272.7178955078125
255 36 Usher Hey Daddy (Daddy's Home) 224.10403442382812
255 37 Kid Cudi / MGMT / Ratatat Pursuit Of Happiness (nightmare) 295.6795349121094


221 62 Boards of Canada 84 Pontiac Dream 229.82484436035156
221 63 Tom Waits Sea Of Love 223.26812744140625
221 64 Philippe Rochard Crumpshit 360.5154724121094
221 65 The Dream Academy Life In A Northern Town (LP Version) 259.2909851074219
221 66 Pantera Planet Caravan (Remastered LP Version) 244.50567626953125
221 67 John Miles Music 351.9472961425781
221 68 Fastway Don't Stop The Fight 259.78729248046875
221 69 Foo Fighters Everlong 249.9913787841797
221 70 Sascha Funke Quiet Please 298.9709777832031
221 71 Apsci Never Give Up 153.25994873046875
221 72 Cute Is What We Aim For Miss Sobriety (Album Version) 171.28443908691406
221 73 Andy Starr Rockin' Rollin' Stone 170.86648559570312
221 74 The Coral Liezah 211.9309844970703
221 75 Jadakiss Pain & Torture 189.67465209960938
221 76 Chet Baker Life 264.4371032714844
221 77 Alex Ubago Que pides tu? 249.36444091796875
221 78 The Black Keys 240 Years Before Your Time 1400.267333984375
221 79 Ill NiÃÂ±o Numb 246.22975158691406
221 80 Billy 

816 12 Snow Patrol Shut Your Eyes 197.09341430664062
816 13 Curtis Mayfield Hey_ Baby_ Give It To Me All 284.9954528808594
816 14 Steve Lukather Jammin' With Jesus 355.73504638671875
816 15 Amy Winehouse Fuck Me Pumps 200.22811889648438
816 16 Modest Mouse Fire It Up 274.6248474121094
816 17 Jel No Solution 217.36444091796875
816 18 Tab Benoit Jambalaya 221.09994506835938
816 19 Old Crow Medicine Show Wagon Wheel 231.73179626464844
816 20 Sisqo Thong Song 253.49179077148438
816 21 Metallica One 447.4248352050781
816 22 Marilyn Manson Angel With The Scabbed Wings 232.09750366210938
816 24 Candy Dulfer Big Girl 244.00933837890625
816 25 Sam Paglia The Day Lo Bianco Left Sicily 106.70974731445312
816 26 John Mayer St. Patrick's Day 321.0966796875
816 27 Van Der Graaf Generator Killer 594.5726318359375
816 28 Guus Meeuwis Laat Mij In Die Waan 219.19302368164062
816 29 Support Lesbiens Fisherman's Friend 195.2648468017578
816 30 Carmen Consoli Piccolo Cesare 288.7571105957031
816 31 The Dea

940 0 Traveling Wilburys End Of The Line 207.5946502685547
177 2 Dwight Yoakam You're The One 239.3072967529297
860 0 Kid Cudi Up Up & Away 227.3432159423828
860 1 Stacie Orrico Stuck (Album Version) 225.35791015625
31 1 Eddie Floyd Guess Who 198.19056701660156
502 0 Angus & Julia Stone Jewels And Gold 262.4256591796875
191 0 Benga Pleasure 314.6444396972656
191 1 Basshunter Why 190.1709747314453
191 2 Kings Of Leon Use Somebody 231.81016540527344
982 1 DHT Feat. EdmÃÂ©e Listen To Your Heart 229.98158264160156
982 2 Bersuit Vergarabat Yo Tomo 212.40118408203125
982 3 La Roux Quicksand 185.65179443359375
982 4 Conjure One Center Of The Sun 300.09423828125
982 5 Graham Colton On Your Side 235.10159301757812
982 6 The Starting Line I'm Real 207.85586547851562
982 7 Jimmy Eat World Drugs Or Me 385.802001953125
982 8 Pull Tiger Tail Hurricanes 239.8820037841797
982 9 OneRepublic Secrets 224.67872619628906
982 10 Hannah Montana The Best Of Both Worlds 174.0011749267578
441 1 BeyoncÃÂ© Get 

807 1 J. Karjalainen & Mustat Lasit Sinisten tÃÂ¤htien alla 336.7440490722656
807 2 Freddie Bruno Updated & Still Hated (The Ball Point Composer Album Version) 230.21669006347656
807 3 Coldplay Fix You 294.1122131347656
807 4 Katrina & The Waves Walking On Sunshine 220.91709899902344
553 0 Disturbed Just Stop (Album Version) 223.4509735107422
553 1 Alliance Ethnik Creil City 265.76934814453125
168 0 Fluke Bermuda 478.92852783203125
683 1 Andre Hazes Droomland (duet met Paul de Leeuw) 206.7064971923828
683 2 Coldplay Shiver 298.7619934082031
768 0 System of a Down Sad Statue 205.79220581054688
768 1 Ghostland Observatory Stranger Lover 224.26077270507812
768 2 Evergreen Terrace Zero 172.381591796875
653 0 Kate Ryan La Promesse (Radio Version) 209.6060791015625
1092 0 Dr Feelgood Riot In Cell Block Number Nine 216.13668823242188
1092 1 Chimaira Everything You Love (Album Version) 377.36444091796875
1092 2 Joshua Radin Someone Else's Life 211.2779083251953
1092 3 Bajaga & Instruktori God